In [ ]:
import numpy as np
from scipy.optimize import root
import plotly.graph_objects as go
from plotly.subplots import make_subplots

# Stability, order and accuracy for non-stiff and stiff equations

## Curtiss and Hirschfelder

We consider the following problem :

\begin{equation} 
\left\{ 
\begin{aligned} 
{\mathrm d}_t u(t) & = k \, \big(cos(t) - u(t)) \big) \quad \text{with } k > 1\\ 
u(t_0)  & = u_0 
\end{aligned} 
\right. 
\end{equation}
and in the following, we will assume $t_0=0$.

### Stiffness

The exact solution is given by :

\begin{equation}
u(t) = \frac{k}{k^2+1} \bigg( k \cos(t) + \sin(t) \bigg) + c_0 \, e^{-k t} \quad
\text{avec} \quad c_0 = \bigg( u_0 -\frac{k}{k^2 + 1} \Big( k \cos(t_0) + \sin(t_0) \Big) \bigg)  e^{-k t_0}   
\end{equation}


In [ ]:
class curtiss_model:

    def __init__(self, k):
        self.k = k

    def fcn(self, t, u) :
        k = self.k
        u_dot = k * (np.cos(t) - u)
        return u_dot

    def sol(self, uini, t0, t):
        k = self.k

        c0 = (uini - (k/(k*k + 1)) * (k*np.cos(t0) + np.sin(t0))) * np.exp(k*t0)
        u = (k/(k*k + 1)) * (k*np.cos(t) + np.sin(t)) +  c0 * np.exp(-k*t)
        return u

In [ ]:
uini = 2.
tini = 0.
tend = 10.
k = 10.

cm = curtiss_model(k)
    
texa = np.linspace(tini, tend, 1000)
uexa = cm.sol(uini, tini, texa)

fig = go.Figure()
fig.add_trace(go.Scatter(x=texa, y=uexa, name='f(x)'))

#create slider
steps = []
for k_i in range(10, 101, 10):
    cm = curtiss_model(k_i)
    step = dict(method="update", label = f"{k_i}", args=[{"x": [texa], "y": [cm.sol(uini, tini, texa)]}])
    steps.append(step)
sliders = [dict(currentvalue={'prefix': 'k = '}, steps=steps)]

fig.update_layout(sliders=sliders, title = 'Quasi-exact solution of the Curtiss and Hirschfelder equation')

fig.show()

## Explicit Euler

The explicit Euler method to solve ${\mathrm d}_t u(t) = f(t,u)$ with $u(0)=u_0$ can be written :

$$
\left\{
\begin{aligned}
& U^0 = u_0 \\
& U^{n+1} = U^n + \Delta t \; f(t^n,U^n) \quad \text{where} \quad \Delta t = t^{n+1} - t^n
\end{aligned}
\right.
$$

In [ ]:
class ode_result:
    def __init__(self, y, t):
        self.y = y
        self.t = t

def forward_euler(tini, tend, nt, yini, fcn):

    dt = (tend-tini) / (nt-1)
    t = np.linspace(tini, tend, nt)

    yini_array = np.array(yini)
    neq = yini_array.size

    y = np.zeros((neq, nt), order='F')
    y[:,0] = yini_array

    for it, tn  in enumerate(t[:-1]):
        yn = y[:,it]
        y[:,it+1] = yn + dt*np.array(fcn(tn, yn))

    return ode_result(y, t)

### Solution and error

In [ ]:
uini = 2.
tini = 0.
tend = 2.
k = 50.

cm = curtiss_model(k)
fcn = cm.fcn

texa = np.linspace(tini, tend, 500)
uexa = cm.sol(uini, tini, texa)

fig = make_subplots(rows=2, cols=1)
fig.add_trace(go.Scatter(x=texa, y=uexa, name='Exact solution'), row=1, col=1)

nt = np.arange(30, 201, 10)
for nt_i in nt:
    sol = forward_euler(tini, tend, nt_i, uini, fcn)
    uexa = cm.sol(uini, tini, sol.t)
    fig.add_trace(go.Scatter(visible=False, x=sol.t, y=sol.y[0], mode='markers+lines', line_dash='dot', 
                             name='Forward Euler'), row=1, col=1)
    fig.add_trace(go.Scatter(visible=False, x=sol.t[1:], y=np.abs(sol.y[0]-uexa)[1:], mode='markers', name='Global Error'), row=2, col=1)
    
i_beg = int(np.argwhere(nt==100))
fig.data[2*i_beg+1].visible = True
fig.data[2*i_beg+2].visible = True

# Create and add slider
steps = []
for i, nt_i in enumerate(nt):
    step = dict(method="update", label = f"{nt_i}",
                args=[{"visible": [el==0 or el==2*i+1 or el==2*i+2 for el in range(len(fig.data))]}])
    steps.append(step)
sliders = [dict(active=i_beg, currentvalue={"prefix": "nt : "}, steps=steps)]

fig.update_yaxes(row=1, col=1)
fig.update_layout(sliders=sliders, title = 'Forward euler', height=800)
fig.show()

### Order

In [ ]:
uini = 2.
tini = 0.
tend = 2.
cm = curtiss_model(k=100)
fcn = cm.fcn

nt = np.array([151, 1501, 15001, 150001])

err_1   = np.zeros_like(nt, dtype=float)
err_2   = np.zeros_like(nt, dtype=float)
err_inf = np.zeros_like(nt, dtype=float)

for i, nt_i in enumerate(nt):
    sol = forward_euler(tini, tend, nt_i, uini, fcn)
    uerr = np.abs(cm.sol(uini, tini, sol.t) - sol.y[0])
    err_1[i]   = np.linalg.norm(uerr,1) / nt_i
    err_2[i]   = np.linalg.norm(uerr) / np.sqrt(nt_i)   
    err_inf[i] = np.linalg.norm(uerr,np.inf) 

dt = (tend-tini)/(nt-1)

fig = go.Figure()
fig.add_trace(go.Scatter(x=dt, y=err_1, name='Norm l1'))
fig.add_trace(go.Scatter(x=dt, y=err_2, name='Norm l2'))
fig.add_trace(go.Scatter(x=dt, y=err_inf, name='Norm linf'))
fig.add_trace(go.Scatter(x=dt, y=dt, mode='lines', line_dash='dot', name='slope 1'))
fig.update_xaxes(type='log', exponentformat='e', title='dt')
fig.update_yaxes(type='log', exponentformat='e', title='error norm')
fig.show()

## Backward Euler

The backward Euler method to solve ${\mathrm d}_t u(t) = f(t,u)$ with $u(0) = u_0$ can be written :

\begin{equation*}
\left\{
\begin{aligned}
& U^0 = u_0 \\
& U^{n+1} = U^n + \Delta t \; f(t^{n+1},U^{n+1}), \qquad \Delta t = t^{n+1} - t^n,
\end{aligned}
\right.
\end{equation*}

In [ ]:
def backward_euler(tini, tend, nt, yini, fcn):

    dt = (tend-tini) / (nt-1)
    t = np.linspace(tini, tend, nt)

    yini_array = np.array(yini)
    neq = yini_array.size

    y = np.zeros((neq, nt), order='F')
    y[:,0] = yini_array

    def g(uip1, *args):
        uip, tip1 = args
        return uip1 - uip - dt*np.array(fcn(tip1, uip1))

    for it, tn  in enumerate(t[:-1]):
        yn = y[:,it]
        y0 = yn + dt*np.array(fcn(tn, yn))
        # solve y[:,it+1] - y[:,it] - dt * fcn(tini + (it+1)*dt, y[:,it+1]) = 0
        sol = root(g, y0, (yn, tn+dt))
        y[:,it+1] = sol.x

    return ode_result(y, t)

### Solution and error

In [ ]:
uini = 2.
tini = 0.
tend = 2.
k = 50.

cm = curtiss_model(k)
fcn = cm.fcn

texa = np.linspace(tini, tend, 500)
uexa = cm.sol(uini, tini, texa)

fig = make_subplots(rows=2, cols=1)
fig.add_trace(go.Scatter(x=texa, y=uexa, name='Exact solution'), row=1, col=1)

nt = np.arange(30, 201, 10)
for nt_i in nt:
    sol = backward_euler(tini, tend, nt_i, uini, fcn)
    uexa = cm.sol(uini, tini, sol.t)
    fig.add_trace(go.Scatter(visible=False, x=sol.t, y=sol.y[0], mode='markers+lines', line_dash='dot', 
                             name='Forward Euler'), row=1, col=1)
    fig.add_trace(go.Scatter(visible=False, x=sol.t[1:], y=np.abs(sol.y[0]-uexa)[1:], mode='markers', name='Global Error'), row=2, col=1)
    
i_beg = int(np.argwhere(nt==100))
fig.data[2*i_beg+1].visible = True
fig.data[2*i_beg+2].visible = True

# Create and add slider
steps = []
for i, nt_i in enumerate(nt):
    step = dict(method="update", label = f"{nt_i}",
                args=[{"visible": [el==0 or el==2*i+1 or el==2*i+2 for el in range(len(fig.data))]}])
    steps.append(step)
sliders = [dict(active=i_beg, currentvalue={"prefix": "nt : "}, steps=steps)]

fig.update_yaxes(row=1, col=1)
fig.update_layout(sliders=sliders, title = 'Forward euler', height=800)
fig.show()

### Order

In [ ]:
uini = 2.
tini = 0.
tend = 2.
cm = curtiss_model(k=100)
fcn = cm.fcn

nt = np.array([151, 1501, 15001, 150001])

err_1   = np.zeros_like(nt, dtype=float)
err_2   = np.zeros_like(nt, dtype=float)
err_inf = np.zeros_like(nt, dtype=float)

for i, nt_i in enumerate(nt):
    sol = backward_euler(tini, tend, nt_i, uini, fcn)
    uerr = np.abs(cm.sol(uini, tini, sol.t) - sol.y[0])
    err_1[i]   = np.linalg.norm(uerr,1) / nt_i
    err_2[i]   = np.linalg.norm(uerr) / np.sqrt(nt_i)   
    err_inf[i] = np.linalg.norm(uerr,np.inf) 

dt = (tend-tini)/(nt-1)

fig = go.Figure()
fig.add_trace(go.Scatter(x=dt, y=err_1, name='Norm l1'))
fig.add_trace(go.Scatter(x=dt, y=err_2, name='Norm l2'))
fig.add_trace(go.Scatter(x=dt, y=err_inf, name='Norm linf'))
fig.add_trace(go.Scatter(x=dt, y=dt, mode='lines', line_dash='dot', name='slope 1'))
fig.update_xaxes(type='log', exponentformat='e', title='dt')
fig.update_yaxes(type='log', exponentformat='e', title='error norm')
fig.show()